In [890]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize

In [891]:
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
#    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
#     layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
#     layer_2 = tf.nn.relu(layer_2)
#    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    out_layer = tf.matmul(layer_1, weights['output']) + biases['output']
    return out_layer

In [892]:
def split_data(file):
    filename = [file]
    record_defaults = [tf.float32] * 75   # Eight required float columns
    dataset = tf.contrib.data.CsvDataset(filename, record_defaults)
    return dataset

In [893]:
train_df = pd.read_csv("Data/formated.csv")
test_df = pd.read_csv("Data/formatedTesting.csv")

train_df.columns = [col.strip() for col in train_df.columns]
test_df.columns = [col.strip() for col in test_df.columns]

# Split the dataframe into features and labels arrays
train_features_array = train_df.iloc[:, :-1].values
train_labels_array = train_df.loc[:, "target Feature"].values
train_labels_binarized = label_binarize(train_labels_array, classes=[0,1,2])
print(train_labels_binarized)
test_features_array = test_df.iloc[:, :-1].values
test_labels_array = test_df.loc[:, "target Feature"].values
test_labels_binarized = label_binarize(test_labels_array, classes=[0,1,2])

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 1 0]
 [1 0 0]
 [0 1 0]]


In [894]:
input_nodes = 74 # maybe 75 for a bias
output_nodes = 3
hLayers = 2
hLayerOne = 37 # half of input nodes
hLayerTwo = 37 # for sigmoid function

learning_rate = 0.01
#epochs = 1 # number of times to iterate over data
batches = 3
train_batch = int(len(train_df)/batches)
test_batch = int(len(test_df)/batches)
print(X)
keep_prob = tf.placeholder("float")
print(keep_prob)
X = tf.placeholder("float", [None, input_nodes])
Y = tf.placeholder("float", [1, output_nodes])
print(Y)

Tensor("Placeholder_278:0", shape=(?, 74), dtype=float32)
Tensor("Placeholder_280:0", dtype=float32)
Tensor("Placeholder_282:0", shape=(1, 3), dtype=float32)


In [895]:
weights = {
    'h1': tf.Variable(tf.random_normal([input_nodes, hLayerOne])),
#     'h2': tf.Variable(tf.random_normal([hLayerOne, hLayerTwo])),
    'output': tf.Variable(tf.random_normal([hLayerOne, output_nodes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([hLayerOne])),
#     'b2': tf.Variable(tf.random_normal([hLayerTwo])),
    'output': tf.Variable(tf.random_normal([output_nodes]))
}

In [896]:
# Construct model
logits = neural_net(X)

# Finding the cost of the algorithm
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

# Finding optimization of the algorithm
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [897]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for batch in range(1):
        avg_cost = 0.0
        x_batches = np.array_split(train_features_array, 6807)
        y_batches = np.array_split(train_labels_binarized, 6807)
        for i in range(6807):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                X: batch_x, 
                                Y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
            if i % 1000 == 0:
                print("Epoch:", '%04d' % (i+1), "cost=", "{:.9f}".format(avg_cost))
                
        print("Optimization Finished!")
        correct_prediction = tf.equal(tf.argmax(X, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#         print("Accuracy:", accuracy.eval({X: test_features_array, Y: test_labels_binarized, keep_prob: 1.0}))

Epoch: 0001 cost= 0.000000000
Epoch: 1001 cost= 4.698298277
Epoch: 2001 cost= 5.934567422
Epoch: 3001 cost= 6.381467345
Epoch: 4001 cost= 6.754971243
Epoch: 5001 cost= 7.102835239
Epoch: 6001 cost= 7.455748852
Optimization Finished!
Tensor("Mean_83:0", shape=(), dtype=float32)
